In [ ]:
import pandas as pd
import numpy as np
import os.path
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import gc

from PIL import Image


from tensorflow import keras 
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow.keras import applications
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers.schedules import ExponentialDecay

In [ ]:
from sklearn.model_selection import StratifiedKFold , KFold ,RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import re
import pickle

# Split Train Images to Train and Validation 

In [ ]:
import glob, shutil, os

class_folder=['apartment','church','house','industrial','mixed_rc','officebuilding','retail']

for i in range(NUM_FOLD):
    
    for c in class_folder:
        
        os.makedirs(f'D:/Nafiseh/building_type/cross_validation_subset_2000/dataset_folds_{i}/images/train/'+c,exist_ok= True)

        file_paths = glob.glob(f'D:/Nafiseh/building_type/cross_validation/dataset_folds_{i}/images/train/'+c+'/*.jpg')
        
        for ff in file_paths:
        
            shutil.copyfile(ff,f'D:/Nafiseh/building_type/cross_validation_subset_2000/dataset_folds_{i}/images/train/'+c+f'/{os.path.basename(ff)}')

In [ ]:
class_folder=['apartment','church','house','industrial','mixed_rc','officebuilding','retail']

In [ ]:
def get_model(IMG_SHAPE):

    #preprocess_input=tf.keras.applications.mobilenet_v2.preprocess_input

    #base_model=tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE+(3,),include_top=False,weights='imagenet')
    
    preprocess_input = tf.keras.applications.vgg16.preprocess_input
    
    base_model = tf.keras.applications.VGG16(input_shape=IMG_SHAPE+(3,), include_top=False, weights='imagenet')

    base_model.trainable=True
    
    #fine_tune_at = 104
    
    for layer in base_model.layers:
        
        layer.trainable = True

    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

    feature_batch_average = global_average_layer(base_model.output)

    num_classes=7
    
    #base_learning_rate=0.0001

    initializer=tf.keras.initializers.VarianceScaling(scale = 1, mode ='fan_avg', distribution = 'truncated_normal')

    prediction_layer=tf.keras.layers.Dense(num_classes,kernel_initializer=initializer, activation = 'softmax')

    prediction_batch=prediction_layer(feature_batch_average)

    inputs=tf.keras.Input(shape= IMG_SHAPE+(3,))

    #x=data_augmentation(inputs)

    x=preprocess_input(inputs)

    #x=base_model(x,training=False)

    x=base_model(x)

    x=global_average_layer(x)

    x=tf.keras.layers.Dropout(0.2)(x)

    outputs=prediction_layer(x)

    model=tf.keras.Model(inputs,outputs)

    #model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                  #optimizer = tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate/10),
                  #metrics=['accuracy'])
            
    #decay_steps = 500
    decay_steps = 234 #int(num training samples/batch size)
    decay_rate = 0.98 #(initial_lr/final_lr)**(1/epochs)
    lr_scheduler = ExponentialDecay(0.001, decay_steps, decay_rate)  
    
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                 optimizer = tf.keras.optimizers.SGD(learning_rate=lr_scheduler),metrics=['accuracy'])
    
    return (model)


In [ ]:
data_kfold = pd.DataFrame()
true_class = pd.DataFrame()

In [ ]:
IMG_SIZE = (224,224)
BATCH_SIZE = 50
EPOCHS = 300
SEED = 1
NUM_FOLD = 5

# Class for measuring training time

In [ ]:
from timeit import default_timer as timer

class TimingCallback(keras.callbacks.Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

In [ ]:
import tensorflow as tf

#Initializing Data Generators
train_datagen = ImageDataGenerator(rescale = 1,
                                   shear_range = 0,
                                   zoom_range = 1,
                                   horizontal_flip = False)

validation_datagen = ImageDataGenerator(rescale = 1)

test_generator = ImageDataGenerator(rescale = 1)

es = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', min_delta = 10**-2, patience =100, restore_best_weights =True)


for j in range(NUM_FOLD): 
    
    #x_train_df = dff.iloc[train_idx]
    
    #x_valid_df = dff.iloc[val_idx]
    
    #j+=1


    #training_set = train_datagen.flow_from_dataframe(dataframe=x_train_df, directory=TRAIN_PATH,
                                                 #x_col="Image", y_col="Class",
                                                 #class_mode="categorical",
                                                 #target_size=IMG_SIZE, batch_size=BATCH_SIZE)
                                                 
    training_set = train_datagen.flow_from_directory(directory = 
                                                     f'D:/Nafiseh/building_type/cross_validation/dataset_folds_{j}/images/train/', target_size = IMG_SIZE, 
    
                                                          color_mode = 'rgb',  batch_size = BATCH_SIZE, 
                                                          class_mode = 'categorical', shuffle =True, seed = 10)                                              
    
    #validation_set = validation_datagen.flow_from_dataframe(dataframe=x_valid_df, directory=TRAIN_PATH,
                                                 #x_col="Image", y_col="Class",
                                                 #class_mode="categorical",
                                                 #target_size=IMG_SIZE, batch_size=BATCH_SIZE)
                                                 
    validation_set = validation_datagen.flow_from_directory(directory = 
                                                            f'D:/Nafiseh/building_type/cross_validation/dataset_folds_{j}/images/valid/', target_size = IMG_SIZE,
                                                            
                                                            color_mode = 'rgb', batch_size = BATCH_SIZE,
                                                            class_mode = 'categorical', shuffle = True, seed = 10)                                               
    
    model_test = get_model(IMG_SIZE)
    
    cb = TimingCallback()
    
    
    
    history = model_test.fit( training_set,
                                        validation_data=validation_set,
                                        epochs = EPOCHS,
                                        steps_per_epoch= training_set.samples // BATCH_SIZE, callbacks = [es,cb]
                                        )
    
    print(cb.logs)
    
    print(f'Training Time for fold {j} is ',sum(cb.logs))
                                        
                                        
    model_test.save(f'D:/Nafiseh/building_type/model_vgg16_training_from_scratch_fold_{j}_SGD_2000_train.h5')
    
    with open(f'D:/Nafiseh/building_type/model_vgg16_history_training_from_scratch_fold_{j}_SGD_2000_train','wb') as model_history:
    
      pickle.dump(history.history, model_history)
    
    
    #test_set = test_generator.flow_from_dataframe(dataframe=train, directory=TRAIN_PATH,
                                                 #x_col="Image",y_col="Class",
                                                 #class_mode=None,
                                                 #target_size=IMG_SIZE)
                                                 
    test_set = test_generator.flow_from_directory(directory = f'D:/Nafiseh/building_type/cross_validation/dataset_folds_{j}/images/test/', target_size = IMG_SIZE,
    
                                                color_mode = 'rgb', batch_size = BATCH_SIZE,
                                                class_mode = 'categorical', shuffle= False) 
                                                
                                                
    true_class[j] = pd.Series(test_set.labels)
                                                
    
    pred= model_test.predict(test_set, test_set.samples // BATCH_SIZE)
    predicted_class_indices=np.argmax(pred,axis=1)
                                       
    data_kfold[j] = pd.Series(predicted_class_indices)
    
    np.save(f'D:/Nafiseh/building_type/data_fold_{j}_training_from_scratch_vgg16_SGD_2000_train.npy',data_kfold[j])
    
    gc.collect()

In [ ]:
from tensorflow.keras.models import load_model
import pandas as pd

NUM_FOLD = 5

IMG_SIZE = (224,224)

BATCH_SIZE = 50

true_class = pd.DataFrame()
data_kfold_2 = pd.DataFrame()


test_generator = ImageDataGenerator(rescale = 1)

for f in range(NUM_FOLD):
    
    model_test = load_model(f'D:/Nafiseh/building_type/model_vgg16_training_10_layers_fold_{f}_SGD_train_2000.h5',compile=False)

    
                                                    
    #test_set = test_generator.flow_from_directory(directory = f'D:/Nafiseh/building_type/cross_validation/dataset_folds_{f}/images/test/', target_size = IMG_SIZE,
    
                                                #color_mode = 'rgb', batch_size = BATCH_SIZE,
                                                #class_mode = 'categorical',shuffle = False) 
            
            
    
    test_set = test_generator.flow_from_directory(directory = f'D:/Nafiseh/building_type/Toronto/My_classes_web_and_google_dataset_2/', target_size = IMG_SIZE,
                                                 
                                                 color_mode = 'rgb', batch_size = BATCH_SIZE, class_mode = 'categorical', shuffle = False)

    true_class[f] = pd.Series(test_set.labels)
                                                
    
    pred= model_test.predict(test_set, test_set.samples // BATCH_SIZE)
    predicted_class_indices=np.argmax(pred,axis=1)
                                       
    data_kfold_2[f] = pd.Series(predicted_class_indices)
    
    #np.save(f'D:/Nafiseh/building_type/model_vgg16_data_fold_{f}_training_0_layers_SGD_2000_train.npy',data_kfold[f])
    
    #np.save(f'D:/Nafiseh/building_type/model_mobilenetv2_training_150_layers_data_fold_{f}_Toronto_dataset_2_label_corrected.npy',data_kfold[f])
    
    

In [ ]:
import shutil
import pandas as pd
from pandas import read_csv
import os

mobilenetv2 = read_csv('D:/Nafiseh/building_type/Toronto/dataset_2_MobilenetV2_fine_tune_150_layers_predictions.csv')

vgg16 = read_csv('D:/Nafiseh/building_type/Toronto/dataset_2_vgg16_training_10_layers_predictions.csv')

E=[]

for _,i in vgg16.iterrows():

    if i['class_ref_2'] != i['class_ref']:
        
        try:
        
            shutil.move('D:\\Nafiseh\\building_type\\Toronto\\My_Classes_web_and_google_dataset_2\\'+i['Image'],'D:\\Nafiseh\\building_type\\Toronto\\My_Classes_web_and_google_dataset_2\\'+i['class_ref_2']+'\\'+i['Image'].split('\\',1)[1])
            
            
        except:
            
            pass
            
    


In [ ]:
#ans = train.copy()
ans_2 = pd.DataFrame(index = np.arange(len(data_kfold)), columns=['Class'])



# Taking The Label with Maximum Occurences

train_datagen = ImageDataGenerator(rescale = 1,
                                   shear_range = 0,
                                   zoom_range = 1,
                                   horizontal_flip = False)


training_set = train_datagen.flow_from_directory(directory = 
                                                 'D:/Nafiseh/building_type/cross_validation/dataset_folds_0/images/train/', target_size = IMG_SIZE, 

                                                      color_mode = 'rgb',  batch_size = BATCH_SIZE, 
                                                      class_mode = 'categorical', shuffle =True, seed = 10)   


labels=(training_set.class_indices)

labels2=dict((v,k) for k,v in labels.items())
import collections 
for i in range(len(data_kfold_2)):
    co = collections.Counter(data_kfold_2.loc[i])
    co = sorted(co.items(),key=lambda x: x[1],reverse=True)
    ans_2.Class.loc[i] = co[0][0]


#ans_array = np.array(ans.Class, dtype=int)

#true_class.iloc[:,0] = true_class.iloc[:,0].replace(5,6)

#true_class.iloc[:,0] = true_class.iloc[:,0].replace(4,5)

#true_class.iloc[:,0] = true_class.iloc[:,0].replace(3,4)

#true_class.iloc[:,0] = true_class.iloc[:,0].replace(2,3)

#true_class.iloc[:,0] = true_class.iloc[:,0].replace(1,2)


# Averaged K-Fold Output
print("Accuracy of K-Fold Method: ",accuracy_score(list(true_class.iloc[:2970,0]), list(ans_2.Class.iloc[:2970])))

# Count Number of Class Occurrences

In [ ]:
num_class = 7

score_2 = np.zeros((data_kfold.shape[0],num_class))

for i in range(data_kfold.shape[0]):
    
    for c in range(num_class):

        score_2[i,c] = np.count_nonzero(np.array(data_kfold)[i,:] == c)/NUM_FOLD

print(score_2)    

# ROC Curve

In [ ]:
from sklearn.preprocessing import LabelBinarizer

from sklearn.metrics import RocCurveDisplay

import matplotlib.pyplot as plt

label_binarizer = LabelBinarizer().fit(list(ans.Class))

label_onehot = label_binarizer.transform(list(ans.Class))

label_binarizer_2 = LabelBinarizer().fit(list(ans_2.Class))

label_onehot_2 = label_binarizer_2.transform(list(ans_2.Class))

label_onehott = [label_onehot, label_onehot_2]

colors = ["darkblue","darkorange"]

models = ['MobilenetV2','VGG16']

scoree =[score, score_2]

fig, ax = plt.subplots(figsize = (10,15))


for l,s,c,m in zip(label_onehott, scoree, colors, models):

    RocCurveDisplay.from_predictions(l.ravel(), s.ravel(), linewidth= 5, name = "{} micro-average OvR".format(m), color = c, ax= ax)

plt.plot([0,1],[0,1],"k--", linewidth = 5, label = "chance level (AUC = 0.5)")

plt.axis('square')

plt.xticks(fontsize=20)

plt.yticks(fontsize=20)

plt.xlabel('False Positive Rate',fontsize= 15)

plt.ylabel('True Positive Rate',fontsize= 15)

plt.legend()

plt.show()

In [ ]:
import pandas as pd

df = pd.DataFrame()

df.loc[:,'Image'] = test_set.filenames

for  i in range(NUM_FOLD):

    df.loc[:,f'fold{i+1}'] = data_kfold[i] 
    
    
    
display(df) 

df.to_csv('D:/Nafiseh/building_type/Toronto/mobilenetv2_prediction_datasset_2_label_corrected.csv')

In [ ]:
import seaborn as sns

from sklearn import preprocessing

import tensorflow as tf

le = preprocessing.LabelEncoder()

le.fit(list(true_class.iloc[:2790,0]))  #train here refers to the data used as test!!

train_categorical = le.transform(list(true_class.iloc[:2790,0]))

pred_categorical = le.transform(ans.Class.iloc[:2790])
    
confusion_matrix=tf.math.confusion_matrix(labels= train_categorical, predictions= pred_categorical).numpy()  

_,ax=plt.subplots(figsize=(25,15))

sns.set (font_scale = 3)

sns.heatmap(confusion_matrix/np.sum(confusion_matrix,axis=1)[:,None], annot=True, fmt='.2%', cmap='Blues',ax=ax)

ax.set_xlabel('Predicted labels',fontweight='bold',fontsize = 20)
ax.set_ylabel('True labels',fontweight='bold',fontsize = 20)
ax.set_title('Confusion Matrix',fontweight='bold',fontsize = 26) 


ax.xaxis.set_ticklabels(['apartment', 'church','house','industrial','mixed \n residential \n commercial',
                        'officebuilding','retail'], fontsize = 18)

ax.yaxis.set_ticklabels(['apartment', 'church','house','industrial','mixed \n residential \n commercial',
                        'officebuilding','retail'], fontsize = 18)

#plt.savefig('D:/Nafiseh/building_type/confusion_matrix_mobilenetv2_label_corrected.png')

plt.show()

# Learning Curves

In [ ]:
import pickle
import matplotlib
import matplotlib.pyplot as plt
from random import randint,seed
import glob, os

NUM_FOLDS = 5


#history_names = ['model_vgg16_history_training_0_layers', 'model_vgg16_history_training_5_layers',
                #'model_vgg16_history_training_l0_layers','model_vgg16_history_training_from_scratch']
    
#history_names = ['0.001','0.0001','0.00001','10-6']   

history_names = ['10-3']
    



#num_fine_tune_layers = ['Transfer Learning','5', '10', 'From Scratch']

#num_fine_tune_layers = ['Learning Rate = 10-3','Learning Rate = 10-4','Learning Rate = 10-5','Learning Rate = 10-6']

num_fine_tune_layers = ['fine_tune_300_layers','fine_tune_250_layers','fine_tune_200_layers',
                       'fine_tune_150_layers','fine_tune_100_layers','fine_tune_50_layers']

num_layers = ['Fine Tune 300 Layers','Fine Tune 250 Layers','Fine Tune 200 Layers','Fine Tune 150 Layers',
             'Fine Tune 100 Layers','Fine Tune 50 Layers']

seed(4)

colors = ['#%06X' % randint(0,0xFFFFFF) for i in range(50)]

matplotlib.style.use('seaborn-v0_8')

fig,ax = plt.subplots(2,1, figsize= (360,280))

fig.subplots_adjust(wspace=0.2)

counter = 0


#matplotlib.rc('xtick', labelsize=80) 
#matplotlib.rc('ytick', labelsize=80) 


for h,file in zip(range(len(history_names)),history_names):
    
    for n in num_fine_tune_layers:
    
        for f in range(NUM_FOLDS):

            counter+=1

            #with open('D:/Nafiseh/building_type/LiDAR/model_histories/'+file+f'_fold_{f}_SGD_train_2000','rb') as pickle_file:
            if n == 'full_features_training_from_scratch':

                with open(f'D:/Nafiseh/building_type/LiDAR/model_histories/shuffled_inception_v3_model_history_lidar_balanced_200_{n}_fold_{f+1}_SGD_initial_lr_{file}','rb') as pickle_file:

                   history = pickle.load(pickle_file)
            else:
                
                with open(f'D:/Nafiseh/building_type/LiDAR/model_histories/shuffled_inception_v3_model_history_lidar_balanced_200_{n}_fold_{f+1}_adam_initial_lr_{file}_dropout_0.8_batch_size_15','rb') as pickle_file:
                
                   history = pickle.load(pickle_file)
                    
            line1, = ax[0].plot(history['loss'], c= colors[counter], linewidth = 20, label = f'Train loss fold {f+1} ({num_layers[h]})' )

            line2, = ax[0].plot(history['val_loss'], c= colors[counter], linewidth = 20, label = f'Validation loss fold {f+1} ({num_layers[h]})')


            ax[0].legend(loc = 'upper center',bbox_to_anchor = (0.5, -0.1),ncol = 5, fontsize = 200)

            line3, = ax[1].plot(history['accuracy'], c= colors[counter], linewidth = 20, label = f'Train Acuuracy fold {f+1} ({num_layers[h]})')

            line4, = ax[1].plot(history['val_accuracy'], c= colors[counter], linewidth = 20, label = f'Validation Accuracy fold {f+1} ({num_layers[h]})')

            ax[1].legend(loc = 'upper center', bbox_to_anchor = (0.5, -0.1), ncol = 5, fontsize = 200)

 
ax[0].set_xlabel('Epochs',weight = 'bold', fontsize = 200)
        
ax[0].set_ylabel('Categorical Cross Entropy', weight = 'bold', fontsize = 200)

ax[0].tick_params(labelsize = 130)

ax[0].grid()       
        
        
ax[1].set_xlabel('Epochs', weight = 'bold', fontsize = 200)
        
ax[1].set_ylabel('Accuracy', weight = 'bold', fontsize = 200) 

ax[1].tick_params(labelsize = 130)

ax[1].grid()

plt.tight_layout()



plt.savefig('D:/Nafiseh/building_type/Building_Type_Manuscript/Figures/Learning curves InceptionV3 LiDAR_4.tif') 

        
plt.show()
            

In [ ]:
!pip install jakteristics

In [ ]:
!pip install laspy[laszip]

In [ ]:
!pip install otb

In [ ]:
from jakteristics import las_utils, compute_features, FEATURE_NAMES

import laspy, glob, os


#las_files = glob.glob('G:/Backedup_ssc_2430_10/MyProject2/Ontario_2015_point_cloud_2/Ontario_2015_point_cloud_2_las/*.las')


las_files = ['G:/Backedup_ssc_2430_10/MyProject2/Ontario_2015_point_cloud_2/Ontario_2015_point_cloud_2_las/1km176290483502015LGTA2015_CPC.las',
             'G:/Backedup_ssc_2430_10/MyProject2/Ontario_2015_point_cloud_2/Ontario_2015_point_cloud_2_las/1km176310483502015LGTA2015_CPC.las']

features = ['eigenvalue_sum', 'omnivariance', 'eigenentropy', 'anisotropy', 'planarity', 'linearity', 'PCA1', 'PCA2', 'surface_variation', 'sphericity', 'verticality']

for f in las_files:
    
    for idx, ff in enumerate(features):

        xyz = las_utils.read_las_xyz(f)

        #building_points = las.points[las.classification == 6] #6:buildings

        features_point_cloud = compute_features(xyz, search_radius=0.15, feature_names=[ff])

        tmp = os.path.basename(f).split('.las')[0]
        
        output_las_file = f'D:/Nafiseh/Building_Footprint_Clip/point_cloud_covariance_features_2/{tmp}_{ff}.las'

        las_utils.write_with_extra_dims(f, output_las_file, features_point_cloud, [ff])

In [ ]:
print(features_point_cloud.shape)

In [ ]:
import otbApplication

HaralickTextureExtraction = otbApplication.Registrry.CreateApplication('HaralickTextureExtraction')

HaralickTextureExtraction.SetParameterString('in','D:/Nafiseh/Building_Footprint_Clip/DSM_2.tif')

HaralickTextureExtraction.SetParameterInt('channels',1)

HaralickTextureExtraction.SetParameterInt('parameters.xrad',3)

HaralickTextureExtraction.SetParameterInt('parameters.yrad',3)

HaralickTextureExtraction.SetParameterString('texture','simple')

HaralickTextureExtraction.SetParameterString('out','D:/Nafiseh/building_type/LiDAR/GLCM_Features.tif')

In [ ]:
def get_key(dic,val):
    
    for key,value in dic.items():
        
        if val == value:
            
           return(key) 

In [ ]:
#!pip install basemap
#!pip install basemap-data-hires
#!pip install contextily
import shapefile as shp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mpl_toolkits.basemap import Basemap
import geopandas as gp
import contextily as cx



sf = shp.Reader('D:/Nafiseh/building_type/Toronto_joined_building_fp_2_wgs84.shp')

sf_gta_boundary = shp.Reader('D:/Nafiseh/building_type/Part_of_GTA_Boundary.shp')

#fields = [f[0] for f in sf.fields[1:]]

#pr = np.array([x['prediction'] for x in sf.iterRecords()])

ref = np.array([x['referenc_1'] for x in sf.iterRecords()])

lat = np.array([x['Latitude'] for x in sf.iterRecords()])

long = np.array([x['Longitude'] for x in sf.iterRecords()])

ref[ref == '#N/A']='apartment'

#pr = pr.astype(int)

colors = ['blue','green','orange','red','darkred','magenta']

ref_color = dict(zip(colors,np.unique(ref)))

ref_ = list(ref_color.values())

c_ = list(ref_color.keys())

#print(pr)

plt.figure(figsize=(100,50))

fig.set_dpi(300)


plt.rcParams['font.family'] = 'Times New Roman'

labels=[]
for r,shape in zip(ref,sf.shapeRecords()):

         x = [i[0] for i in shape.shape.points[:]]

         y = [i[1] for i in shape.shape.points[:]]
 
         if r in labels:   

             plt.plot(x,y,c=get_key(ref_color,r), linewidth=100, alpha=0.7)
                
         else:
            
             plt.plot(x,y,c=get_key(ref_color,r),linewidth=100,label=r,alpha=0.7)
                
         labels.append(r)        
                
    
for shape in sf_gta_boundary.shapeRecords():

     x1 = [i[0] for i in shape.shape.points[:]]
        
     y1 = [i[1] for i in shape.shape.points[:]] 
    
     plt.plot(x1,y1, linewidth=25, c='black', label='Part of GTA Boundary',alpha=0.7) 
        
         
ax = plt.gca()

cx.add_basemap(ax, crs=4326)

plt.xticks(fontsize=150)

plt.yticks(fontsize=150)

plt.legend(prop={'size':100})

plt.savefig('D:/Nafiseh/building_type/Building_Type_Manuscript/Figures/Toronto_Map_2.tif',bbox_inches='tight')

plt.show()


In [ ]:
#!pip install basemap
#!pip install basemap-data-hires
#!pip install contextily
import shapefile as shp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mpl_toolkits.basemap import Basemap
import geopandas as gp
import contextily as cx



sf = shp.Reader('D:/Nafiseh/building_type/Toronto_joined_building_fp_2_wgs84.shp')

sf_gta_boundary = shp.Reader('D:/Nafiseh/building_type/Part_of_GTA_Boundary.shp')

#fields = [f[0] for f in sf.fields[1:]]

pr = np.array([x['prediction'] for x in sf.iterRecords()])

ref = np.array([x['referenc_1'] for x in sf.iterRecords()])

lat = np.array([x['Latitude'] for x in sf.iterRecords()])

long = np.array([x['Longitude'] for x in sf.iterRecords()])

ref[ref == '#N/A']='apartment'

pr[pr == '#N/A']='0'

#pr = pr.astype(int)

colors = ['blue','green','orange','red','darkred','magenta']

colors_2 = ['blue','cyan','green','orange','red','darkred','magenta']

ref_color = dict(zip(colors,np.unique(ref)))

p_color = dict(zip(colors_2, np.unique(pr)))

p_num_desc = dict(zip(np.unique(pr),np.array(['apartment','church','house','industrial','mixed r/c','office building','retail'])))

ref_ = list(ref_color.values())

c_ = list(ref_color.keys())

#print(pr)

fig,ax=plt.subplots(2,1,figsize=(200,100))

fig.set_dpi(300)


plt.rcParams['font.family'] = 'Times New Roman'



labels=[]
for r,shape in zip(ref,sf.shapeRecords()):

         x = [i[0] for i in shape.shape.points[:]]

         y = [i[1] for i in shape.shape.points[:]]
 
         if r in labels:   

             ax[0].plot(x,y,c=get_key(ref_color,r), linewidth=100, alpha=0.7)
                
         else:
            
             ax[0].plot(x,y,c=get_key(ref_color,r),linewidth=100,label=r,alpha=0.7)
                
         labels.append(r)        
                
    
for shape in sf_gta_boundary.shapeRecords():

     x1 = [i[0] for i in shape.shape.points[:]]
        
     y1 = [i[1] for i in shape.shape.points[:]] 
    
     ax[0].plot(x1,y1, linewidth=25, c='black', label='Part of GTA Boundary',alpha=0.7) 

cx.add_basemap(ax[0], crs=4326)

#ax[0].set_title('Ground Truth Map',fontsize=200)

ax[0].set_xlabel('(a)', fontsize =150, weight='bold')

ax[0].tick_params(axis='x', labelsize=150)

ax[0].tick_params(axis='y', labelsize=150)

ax[0].legend(prop={'size':100})

labels_2=[]

for p,shape in zip(pr,sf.shapeRecords()):
    
    x2 = [i[0] for i in shape.shape.points[:]]
    
    y2 = [i[1] for i in shape.shape.points[:]]
    
    if p in labels_2:
    
        ax[1].plot(x2,y2, c=get_key(p_color,p),linewidth=100,alpha=0.7)
        
    else:
        
        ax[1].plot(x2,y2,c=get_key(p_color,p),linewidth=100,label=p_num_desc[p],alpha=0.7)
        
    labels_2.append(p)
    
for shape in sf_gta_boundary.shapeRecords():
    
     x3 = [i[0] for i in shape.shape.points[:]]
        
     y3 = [i[1] for i in shape.shape.points[:]]
    
     ax[1].plot(x3,y3,linewidth=25,c='black',label='Part of GTA Boundary',alpha=0.7)    
    
cx.add_basemap(ax[1], crs=4326) 

#ax[1].set_title('Predicted Map',fontsize=200)


ax[1].set_xlabel('(b)', fontsize =150, weight='bold')

ax[1].tick_params(axis='x', labelsize=150)

ax[1].tick_params(axis='y', labelsize=150)

ax[1].legend(prop={'size':100})

#plt.tight_layout()

fig.savefig('D:/Nafiseh/building_type/Building_Type_Manuscript/Figures/Toronto_Map_4.tif',bbox_inches='tight')

plt.show()


In [ ]:
pr = np.array([x['prediction'] for x in sf.iterRecords()])

pr[pr == '#N/A']='0'

print(pr)

In [ ]:
p_num_desc = dict(zip(np.unique(pr),np.array(['apartment','church','house','industrial','mixed r/c','office building','retail'])))


print(p_num_desc)